In [ ]:
import requests
import csv
from ltr.data import CorpusApi, Config
import numpy as np

In [ ]:
queries = CorpusApi.getValidationQueriesAsDict()

In [ ]:
import re

for i, key in enumerate(list(queries.keys())[:]):
    try:
         # get query and clean the query string
        query = queries[key]
        query = re.sub('[^\w\s]', '', query)
        
        # get result from solr
        request = {
            "fields": "id, score",
            "limit": 500,
            "params": {
                "qf": "title, headings, body, _copy_all_",
                "defType": "dismax",
                "q": query
            }
        }
        
        response = requests.post(f'http://localhost:8983/solr/thesis-ltr/select', json=request)
        
        queryResult = [doc for doc in response.json()["response"]["docs"]]
        
        # write resultfile for trec eval
        with open('evaluation/bm25_title_headings_body_eval.results', 'a') as file:
            for j, doc in enumerate(queryResult):
                line = " ".join([str(key), "Q0", str(doc['id']), str(j + 1), str(doc['score']), "STANDARD"])
                file.write(line)
                file.write('\n')
        
        print(f'Processed queries: {i + 1}/{len(list(queries.keys()))}')
    except KeyError as err:
        # ignore the queries that are not in the validation queries list
        pass